In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('csv/online_sales.csv')

/tmp/ipykernel_39851/2166572984.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('csv/online_sales.csv')


In [9]:
df.isna().sum()

order_id            0
order_date          0
status              0
item_id             0
sku                 0
qty_ordered         0
price               0
value               0
discount_amount     0
total               0
category            0
payment_method      0
bi_st               0
cust_id             0
year                0
month               0
ref_num             0
Name Prefix         0
First Name          0
Middle Initial      0
Last Name           0
Gender              0
age                 0
full_name           0
E Mail              0
Customer Since      0
SSN                 0
Phone No.           0
Place Name          0
County              0
City                0
State               0
Zip                 0
Region              0
User Name           0
Discount_Percent    0
age_group           0
dtype: int64

Today we will analyze online sales data to understand market and customer trends. We work for an imaginary online store in the US. The goal is to extract insights from the dataset online_sales in order to give recommendations to the company on new effective marketing campaigns.


Load the dataset, understand the data, make visualizations, clean the data (think which transactions are counted as valid to your project) and make all the necessary data transformation and aggregations to answer the following questions (communicate your results with visualizations):


Which category gives the highest sales?  
Which categories are bought the most from men and which from women?  
Which month of the year has the highest revenue? Can you guess why?  
Which are the top five states that generate the highest revenue?  
Which category is the highest in terms of revenue for each state? (Hard!)  
Which category gives discounts more often? Visualize the top 5 of them.  
Which age segment spends the most?  
For the category with the highest revenue, plot the revenue per week for CA.  
What is the average item count per order in each category?  
Which two days of the week have the highest number of transactions?  
If you wanted to launch an effective marketing campaign, in which category, age, gender, time of the year and days of the week would be to maximize its efficiency? Are you considering offering products at a discount and why?